In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

In [ ]:
from tda.experiments.lid.lid_binary import logger, run_experiment, Config, Architecture, get_feature_datasets
from tda.models.architectures import get_architecture, mnist_lenet
from tda.models import get_deep_model
from tda.models.datasets import Dataset

In [ ]:
config = Config(
    epochs=50,
    dataset="MNIST",
    architecture=mnist_lenet.name,
    train_noise=0.0,
    nb_batches=10,
    attack_type="FGSM",
    noise=0.0,
    num_iter=1,
    batch_size=100,
    number_of_nn=20
)

In [ ]:
dataset = Dataset(name=config.dataset)

logger.info(f"Getting deep model...")
archi: Architecture = get_deep_model(
    num_epochs=config.epochs,
    dataset=dataset,
    architecture=get_architecture(config.architecture),
    train_noise=config.train_noise
)

datasets = dict()

for epsilon in [0.01, 0.025, 0.05, 0.1, 0.4, 1.0]:
    ds_train, ds_test = get_feature_datasets(
        config=config,
        epsilon=epsilon,
        dataset=dataset,
        archi=archi
    )
    datasets[epsilon] = (ds_train, ds_test)

In [ ]:
datasets[0.4][0]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

scaler = StandardScaler()

all_aucs = dict()
all_coefs = dict()

for epsilon in datasets:
    train_ds, test_ds = datasets[epsilon]
    
    detector = LogisticRegression(
        fit_intercept=True,
        verbose=1,
        tol=1e-5,
        max_iter=1000,
        solver='lbfgs'
    )
    
    ds_train_scaled = scaler.fit_transform(train_ds.iloc[:, :-1])

    detector.fit(X=ds_train_scaled, y=train_ds.iloc[:, -1])
    coefs = list(detector.coef_.flatten())
    logger.info(f"Coefs of detector {coefs}")
    
    ds_test_scaled = scaler.transform(test_ds.iloc[:, :-1])

    test_predictions = detector.predict_proba(X=ds_test_scaled)[:, 1]
    auc = roc_auc_score(y_true=test_ds.iloc[:, -1], y_score=test_predictions)
    logger.info(f"AUC is {auc}")

    all_aucs[epsilon] = auc
    all_coefs[epsilon] = coefs

logger.info(all_aucs)

In [ ]:
epsilons = sorted(all_aucs.keys())

plt.figure()
plt.xlabel('Epsilon')
plt.ylabel('LID AUC')
plt.xscale('log')
plt.plot(
    epsilons, [all_aucs[eps] for eps in epsilons]
)